### 코드 순서
- [0️⃣ Initialize notebook variables](#0)
- [🧪 Azure OpenAI를 직접 호출하여 On Your Data에서 응답을 리턴하는지 테스트](#requests)
- [🧪 API Management를 직접 HTTP를 호출하여 API 테스트](#requests)

### 사전 준비 사항
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- 리소스 그룹 명, Azure OpenAI에서 배포한 deployment 이름 (예: gpt-4o), OpenAI API 버전 (예:2024-02-01)을 입력합니다.
- API Gateway URL과 Subscription Key를 입력합니다.
- Subscription Key는 Azure Portal의 API Management 리소스에서 APIs->Subscriptions->Built-in all-access subscription의 Primary key를 입력합니다.

In [ ]:
import os
import json
import datetime
import requests

resource_group_name = "{resource-group-name}" # 이전 단계에서 생성한 리소스 그룹 이름을 입력합니다.
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"
openai_model_name = "gpt-4o"
apim_resource_gateway_url = "{apim-endpoint}" # 이전 단계에서 생성한 APIM 리소스의 게이트웨이 URL을 입력합니다.
apim_subscription_key = "{apim-subscriptoin-key}" # 이전 단계에서 생성한 APIM 리소스의 구독 키를 입력합니다.


<a id='requests'></a>
### 🧪 Azure OpenAI를 직접 호출하여 On Your Data에서 응답을 리턴하는지 테스트

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = "{aoai_endpoint}" # Azure OpenAI 리소스의 엔드포인트를 입력합니다.
api_key = "{aoai_api_key}"
deployment = "gpt-4o"

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "서울교대부초에 원서 접수를 할 수 있는 조건은 무엇인가요?",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": "{ai_search_endpoint}",
                    "index_name": "s-index",
                    "authentication": {
                        "type": "api_key",
                        "key": "{ai_search_key}",
                    }
                }
            }
        ],
    }
)

print(completion.model_dump_json(indent=2))

{
  "id": "74951e35-1786-42ed-96e3-647480b374e9",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The requested information is not available in the retrieved data. Please try another query or topic.",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": null,
        "end_turn": true,
        "context": {
          "citations": [
            {
              "content": "본교에 특수교육대상자로 신청 접수 후 사전 심사를 통해 추첨대상자로 통보된 어린이\n - 특수교육대상학생 전형 공고(2024.7.26..(월) 이후): 개별 문자 발송\n - 인터넷 원서 접수(2024.10.28..특수교육대상자가 일반전형으로 당첨된 경우에는 입학 후 본교의 특수교육대상자로 \n\n선정될 수 없음, ‘특수교육법 27조 ①항 특수학급당 인원수(6명)기준’에 의함\n   (본교는 특수학급을 1개 운영하며 증설이 불가함)\n ※ 특별전형(특수교육대상자)에 원서 접수한 어린이는 일반전형과 특별전형(국가보훈대상자)  \n\n  에 이중지원할 수 없으며, 이중지원 시 당첨 취소 처리함\n국가보훈대상자(모집인원 3명)\n\n1..원서 접수 개시일(2024.10.28.).독립유공자의 경우만 (손)자녀 해당\n※ 특별전형(국가보훈대상자)에 원서 접수한 어린이는 일반전형과 

<a id='requests'></a>
### 🧪 API Management를 직접 HTTP를 호출하여 API 테스트

In [ ]:
import json

url = "https://apim-jl-apr01.azure-api.net/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01" #예: https://apim-jl-apr01.azure-api.net/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01
api_runs = []
for i in range(1):
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": "{ai-search-endpoint}",
                    "index_name": "s-index",
                    "authentication": {
                        "type": "api_key",
                        "key": "{ai-search-key}",
                    }
                }
            }
        ],
    }
    
    messages={
        "messages":[
            {"role": "system", "content": "You are an AI assistant that helps people find information."},
            {"role": "user", "content": "서울교대부초와 중대부초 중에 원서 접수가 먼저 마감되는 곳은 어디인가요?"}
        ]
    }

    json_messages = messages | extra_body

    #response = requests.post(url, headers = {'Ocp-Apim-Subscription-Key': 'c1cb740038ff4f8095815d4967cd33e6', 'work-type': 'work'}, json = json_messages)
    #response = requests.post(url, headers = {'Ocp-Apim-Subscription-Key': 'c1cb740038ff4f8095815d4967cd33e6', 'work-type': 'work'}, json = json_messages)
    response = requests.post(url, headers = {'Ocp-Apim-Subscription-Key': 'c1cb740038ff4f8095815d4967cd33e6'}, json = json_messages)

    if (response.status_code == 200):
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "✅")
        data = json.loads(response.text)
        total_tokens = data.get("usage").get("total_tokens")
        print("💬 ", data.get("choices")[0].get("message").get("content"))
    else:
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "⛔")
        print(response.text)
        total_tokens = 0
    
    print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # 이 헤더를 통하여 어느 Region의 Azure OpenAI 서비스에서 응답을 리턴했는지 파악할 수 있음
    s = response.content.decode("utf-8")
    #print(s) # 응답의 내용을 확인할 수 있음

    api_runs.append((total_tokens, response.status_code))


▶️ Run:  1 status code:  200 ✅
💬  서울교대부초의 원서 접수는 2024년 10월 28일에 시작됩니다 [doc1]. 중대부초의 원서 접수 마감일에 대한 정보는 제공된 문서에서 찾을 수 없습니다. 따라서 서울교대부초의 원서 접수 시작일만 확인할 수 있습니다. 중대부초의 정확한 마감일을 확인하기 위해서는 추가적인 자료가 필요합니다.
x-ms-region: West US
